In [1]:
# !pip install --no-cache-dir transformers==4.5.1 sentencepiece
# !pip install -U sentence_transformers
# !pip install plotly==4.9.0
# !pip install wmd

In [2]:
import sys
import re
from collections import defaultdict
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances
from scipy.spatial.distance import euclidean, pdist, squareform
from sklearn import manifold          #use this for MDS computation
import pandas as pd
import numpy as np

#visualization libs
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt


#Used to calculation of word movers distance between sentence
from collections import Counter

#Library to calculate Relaxed-Word Movers distance
from wmd import WMD
from wmd import libwmdrelax

In [3]:
 model_path = './output/sentence-transformers_stsb-xlm-r-multilingual-2021-05-11_10-04-26/'

In [4]:
from transformers import AutoTokenizer, AutoModel, AutoConfig
from sentence_transformers import util
import torch

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    print('Token embeddings: ', token_embeddings.shape)
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    print(sum_embeddings.shape)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def second_last_mean_pooling(model_output, attention_mask):
    hidden_layers = model_output[2]
    token_embeddings = torch.stack([hidden_layers[1], hidden_layers[-1]]).mean(0)
    print('Token embeddings: ', token_embeddings.shape)
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    print(sum_embeddings.shape)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def make_attention_mask_without_special_token(attention_mask):
    
    attention_mask_without_special_tok = attention_mask.clone().detach()
    
    #set the CLS token index to 0 for all sentences 
    attention_mask_without_special_tok[:,0] = 0

    #get sentence lengths and use that to set those indices to 0 for each length
    #essentially, the last index for each sentence, which is the SEP token
    sent_len = attention_mask_without_special_tok.sum(1).tolist()

    #column indices to set to zero
    col_idx = torch.LongTensor(sent_len)
    #row indices for all rows
    row_idx = torch.arange(attention_mask.size(0)).long()
    
    #set the SEP indices for each sentence token to zero
    attention_mask_without_special_tok[row_idx, col_idx] = 0
    return attention_mask_without_special_tok


def mean_pooling_no_spec_tokens(model_output, attention_mask):
    attention_mask = make_attention_mask_without_special_token(attention_mask)
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def second_last_mean_pooling_no_spec_tokens(model_output, attention_mask):
    attention_mask = make_attention_mask_without_special_token(attention_mask)
    hidden_layers = model_output[2]
    token_embeddings = torch.stack([hidden_layers[1], hidden_layers[-1]]).mean(0)
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [5]:
config = AutoConfig.from_pretrained(model_path, output_hidden_states=True, output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path, config=config)
# model.eval()

base_config = AutoConfig.from_pretrained('sentence-transformers/stsb-xlm-r-multilingual', output_hidden_states=True, output_attentions=True)
base_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/stsb-xlm-r-multilingual', config=base_config)
base_model = AutoModel.from_pretrained('sentence-transformers/stsb-xlm-r-multilingual')
# base_model.eval()


Some weights of XLMRobertaModel were not initialized from the model checkpoint at ./output/sentence-transformers_stsb-xlm-r-multilingual-2021-05-11_10-04-26/ and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
#Sentences we want sentence embeddings for
# sentences = ['Quiero hincarle el diente a tu culo redondo','Esta noche voy a cogerte bien', 'He amado, he llorado, he besado, me he entregado']
sentences = ['Eso que me atrae de ti no sé lo que es No sé lo que tienes pero quiero que me des Es inevitable, tu figura me llama Luce sana, tu dulce mirada me reclama Quiero conocerte a fondo Contarte las cosas que me ponen cachondo Besarte como un adolescente ardiente Quiero hincarle el diente a tu culo redondo Sexo en la primera mirada Era el postre que se adivinaba',
            'Esta noche voy a cogerte bien nos iremos a casa de tu papá llamaré a la puerta, nos esconderemos tiraremos piedras para no quedar bien Y cuando piensen quién ha sido le diremos que no, no han sido tus amigos, allí nadie quedó. ya no sabes qué hacer Adiós papá, adiós papá, consíguenos un poco de dinero más (x2) más dinero',
             'He amado, he llorado, he besado, me he entregado He sido mala y hasta cruel Sin pensar en creer en otro hombre en este mundo Pensé que como mujer iba ya a tener bastante No más miedos, no más hombres en mis llantos No más sueños destrozados, no más días sin amar No más sentirme atrapada, vivir sin aire Pero mi plan ha cambiado, he perdido el control']

#Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, add_special_tokens=True, max_length=128, return_tensors='pt')
base_encoded_input = base_tokenizer(sentences, padding=True, truncation=True, max_length=128, add_special_tokens=True, return_tensors='pt')
#Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)
    print(model_output.keys())
    print(model_output[2][0].shape)
    base_model_output = base_model(**base_encoded_input)



odict_keys(['last_hidden_state', 'pooler_output', 'hidden_states', 'attentions'])
torch.Size([3, 93, 768])


In [8]:
#Perform pooling. In this case, mean pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
base_sentence_embeddings = mean_pooling(base_model_output, base_encoded_input['attention_mask'])
second_last_sentence_embeddings = second_last_mean_pooling(model_output, encoded_input['attention_mask'])

Token embeddings:  torch.Size([3, 93, 768])
torch.Size([3, 768])
Token embeddings:  torch.Size([3, 93, 768])
torch.Size([3, 768])
Token embeddings:  torch.Size([3, 93, 768])
torch.Size([3, 768])


In [9]:
def cos_sim(sentences, sentence_embeddings):
    cosine_scores = util.pytorch_cos_sim(sentence_embeddings, sentence_embeddings)

    #Find the pairs with the highest cosine similarity scores
    pairs = []
    for i in range(len(cosine_scores)-1):
        for j in range(i+1, len(cosine_scores)):
            pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})

    #Sort scores in decreasing order
    pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)

    for pair in pairs[0:10]:
        i, j = pair['index']
        print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], pair['score']))

In [10]:
print('Cosine similarity - with special tokens')
print('Original model')
cos_sim(sentences, base_sentence_embeddings)
print('Our model')
cos_sim(sentences, sentence_embeddings)
print('taking avarage of second and last layer of our model')
cos_sim(sentences, second_last_sentence_embeddings)

Cosine similarity - with special tokens
Original model
Esta noche voy a cogerte bien nos iremos a casa de tu papá llamaré a la puerta, nos esconderemos tiraremos piedras para no quedar bien Y cuando piensen quién ha sido le diremos que no, no han sido tus amigos, allí nadie quedó. ya no sabes qué hacer Adiós papá, adiós papá, consíguenos un poco de dinero más (x2) más dinero 		 He amado, he llorado, he besado, me he entregado He sido mala y hasta cruel Sin pensar en creer en otro hombre en este mundo Pensé que como mujer iba ya a tener bastante No más miedos, no más hombres en mis llantos No más sueños destrozados, no más días sin amar No más sentirme atrapada, vivir sin aire Pero mi plan ha cambiado, he perdido el control 		 Score: 0.6043
Eso que me atrae de ti no sé lo que es No sé lo que tienes pero quiero que me des Es inevitable, tu figura me llama Luce sana, tu dulce mirada me reclama Quiero conocerte a fondo Contarte las cosas que me ponen cachondo Besarte como un adolescente ar

In [10]:
#Perform pooling. In this case, mean pooling
sentence_embeddings = mean_pooling_no_spec_tokens(model_output, encoded_input['attention_mask'])
base_sentence_embeddings = mean_pooling_no_spec_tokens(base_model_output, base_encoded_input['attention_mask'])
second_last_sentence_embeddings = second_last_mean_pooling_no_spec_tokens(model_output, encoded_input['attention_mask'])

In [11]:
print('Cosine similarity - without special tokens')
print('Original model')
cos_sim(sentences, base_sentence_embeddings)
print('Our model')
cos_sim(sentences, sentence_embeddings)
print('taking avarage of second and last layer of our model')
cos_sim(sentences, second_last_sentence_embeddings)

Cosine similarity - without special tokens
Original model
Esta noche voy a cogerte bien 		 He amado, he llorado, he besado, me he entregado 		 Score: 0.3535
Quiero hincarle el diente a tu culo redondo 		 Esta noche voy a cogerte bien 		 Score: 0.2576
Quiero hincarle el diente a tu culo redondo 		 He amado, he llorado, he besado, me he entregado 		 Score: 0.2183
Our model
Quiero hincarle el diente a tu culo redondo 		 Esta noche voy a cogerte bien 		 Score: 0.6919
Esta noche voy a cogerte bien 		 He amado, he llorado, he besado, me he entregado 		 Score: 0.4559
Quiero hincarle el diente a tu culo redondo 		 He amado, he llorado, he besado, me he entregado 		 Score: 0.4383
taking avarage of second and last layer of our model
Quiero hincarle el diente a tu culo redondo 		 Esta noche voy a cogerte bien 		 Score: 0.6392
Quiero hincarle el diente a tu culo redondo 		 He amado, he llorado, he besado, me he entregado 		 Score: 0.4705
Esta noche voy a cogerte bien 		 He amado, he llorado, he be